In [ ]:
_coco = COCO(train_coco)

img_id = 500
ann_ids = _coco.getAnnIds(imgIds=img_id)
print("ann_ids\n", ann_ids)
coco_annotation = _coco.loadAnns(ann_ids)
print("coco_annotation\n", coco_annotation)
_coco.loadImgs(img_id)[0]['file_name']

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
ann_ids
 [2926, 2927, 2928, 2929, 2930, 2931, 2932, 2933, 2934, 2935, 2936, 2937, 2938, 2939, 2940, 2941, 2942, 2943, 2944, 2945, 2946, 2947, 2948, 2949, 2950, 2951, 2952, 2953, 2954, 2955]
coco_annotation
 [{'id': 2926, 'image_id': 500, 'category_id': 1, 'bbox': [2411, 1908, 12.69, 14.67], 'area': 186.162, 'segmentation': [], 'iscrowd': 0}, {'id': 2927, 'image_id': 500, 'category_id': 1, 'bbox': [2436, 1912, 6.64, 9.76], 'area': 64.806, 'segmentation': [], 'iscrowd': 0}, {'id': 2928, 'image_id': 500, 'category_id': 1, 'bbox': [2371, 1921, 17.89, 12.67], 'area': 226.666, 'segmentation': [], 'iscrowd': 0}, {'id': 2929, 'image_id': 500, 'category_id': 1, 'bbox': [2351, 1924, 17.33, 14.72], 'area': 255.098, 'segmentation': [], 'iscrowd': 0}, {'id': 2930, 'image_id': 500, 'category_id': 1, 'bbox': [2336, 1938, 14.93, 14.39], 'area': 214.843, 'segmentation': [], 'iscrowd': 0}, {'id': 2931, 'image_id': 500, 'c

'DJI_0106_JPG.rf.537bc48bfdacc8fdf4c2e45e1e2ab651.jpg'

In [ ]:
import pickle


with open('result.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data[1]

{'scale_factor': (0.26737967914438504, 0.26737967914438504),
 'img_path': '/content/wind-turbine-damage-challenges/Wind Turbine damage/test/20240120_163730-0-_jpg.rf.7139d077e629ae78b03081e5ca84dd98.jpg',
 'img_id': 1,
 'pad_shape': (800, 800),
 'batch_input_shape': (800, 800),
 'ori_shape': (2992, 2992),
 'img_shape': (800, 800),
 'pred_instances': {'labels': tensor([0, 0, 0]),
  'bboxes': tensor([[1811.5305,  141.6571, 1874.7821,  220.4477],
          [2011.0117, 1290.4733, 2056.0708, 1378.9531],
          [1801.2488,   12.1943, 1985.1924,  324.6060]]),
  'scores': tensor([0.4198, 0.1335, 0.0866])}}

In [ ]:
import pandas as pd


df = pd.DataFrame(index=['image_id'], columns=['objects'])
df

,objects
image_id,NaN


In [ ]:
new_row = pd.DataFrame({"image_id": ["555"], "objects": [28]})
df = pd.concat([df, new_row], ignore_index=True)
df

,"image_id, objects",image_id,objects
0,NaN,555,28.0
1,NaN,555,28.0
2,NaN,555,28.0


In [ ]:
test_names = os.listdir('/content/Wind Turbine damage/test')
test_names[0]

'20240120_155211_mp4-0079_jpg.rf.ed1b222a0cf341a04f27bcdc26a65687.jpg'

In [ ]:
import numpy as np

data[1]['pred_instances']['labels'].tolist()

[0, 0, 0]

In [ ]:
data[0]['img_path'].split('/')[-1]

'20240120_155211_mp4-0149_jpg.rf.45490f1c8ade5580d1279140588b187b.jpg'

In [ ]:
data[]['pred_instances']

{'labels': tensor([0, 0, 0]),
 'bboxes': tensor([[1811.5305,  141.6571, 1874.7821,  220.4477],
         [2011.0117, 1290.4733, 2056.0708, 1378.9531],
         [1801.2488,   12.1943, 1985.1924,  324.6060]]),
 'scores': tensor([0.4198, 0.1335, 0.0866])}

In [ ]:
data[1]['pred_instances']['labels'][0]

tensor(0)

In [ ]:
defect_names = {
    0: "corrosion",
    1: "lightning",
    2: "lightning receptor",
    3: "missing teeth",
    4: "patch"
}


test_names = os.listdir('/content/Wind Turbine damage/test')

pred_dict = {}

for pred in data:
    img_name = pred['img_path'].split('/')[-1]

    pred_insts = pred['pred_instances']
    defects_len =  len(pred_insts['labels'])
    cur_defects = []

    for i in range(defects_len):
        defect_name = defect_names[int(pred_insts['labels'][i])]
        xmin = float(pred_insts['bboxes'][i][0])
        ymin = float(pred_insts['bboxes'][i][1])
        width = float(pred_insts['bboxes'][i][2] - xmin)
        height =  float(pred_insts['bboxes'][i][3] - ymin)
        score = float(pred_insts['scores'][i])
        cur_defects.append([defect_name, xmin, ymin, width, height, score])

    pred_dict[img_name] = cur_defects


ex_sub = pd.read_csv('/content/sample_submission_with_errors.csv')
df_submit = ex_sub.copy()

for i, row in df_submit.iterrows():
    cur_name = row['image_id']
    upd_val = pred_dict[cur_name]
    df_submit.at[i, 'objects'] = upd_val

df_submit.to_csv('submit_i_hope00.csv', index=False, )

In [ ]:
pred_dict

{'20240120_155211_mp4-0149_jpg.rf.45490f1c8ade5580d1279140588b187b.jpg': [['corrosion',
   61.083805084228516,
   126.05473327636719,
   22.05996322631836,
   69.07562255859375,
   0.06213033199310303]],
 '20240120_163730-0-_jpg.rf.7139d077e629ae78b03081e5ca84dd98.jpg': [['corrosion',
   1811.530517578125,
   141.65711975097656,
   63.2515869140625,
   78.79058837890625,
   0.41983306407928467],
  ['corrosion',
   2011.01171875,
   1290.4732666015625,
   45.05908203125,
   88.4798583984375,
   0.13354508578777313],
  ['corrosion',
   1801.248779296875,
   12.194334030151367,
   183.943603515625,
   312.4117126464844,
   0.08662845939397812]],
 'DJI_0749-1-_JPG.rf.356be7f5b3ec7e31c5594426b6aba5eb.jpg': [['corrosion',
   3676.322021484375,
   1461.102294921875,
   159.084716796875,
   182.4454345703125,
   0.9788091778755188],
  ['corrosion',
   1662.02734375,
   1241.9427490234375,
   65.717041015625,
   63.133056640625,
   0.8613044023513794],
  ['corrosion',
   3718.23876953125,
   14

In [ ]:
df_dic = {'image_id': img_names, 'objects': defects}

df = pd.DataFrame(df_dic)
df

,image_id,objects
0,20240120_155211_mp4-0149_jpg.rf.45490f1c8ade55...,"[[corrosion, 61.083805084228516, 126.054733276..."
1,20240120_163730-0-_jpg.rf.7139d077e629ae78b030...,"[[corrosion, 1811.530517578125, 141.6571197509..."
2,DJI_0749-1-_JPG.rf.356be7f5b3ec7e31c5594426b6a...,"[[corrosion, 3676.322021484375, 1461.102294921..."
3,20240120_164348_mp4-4909_jpg.rf.c2880db9ff2ebe...,[]
4,DJI_0764-1-_JPG.rf.dda28e9da6a04fb32538af96498...,[]
...,...,...
146,20240120_163644-0-_jpg.rf.968f7d72ae7be6ef8733...,[]
147,20240120_155211_mp4-0160_jpg.rf.94f5e7408084ab...,"[[corrosion, 239.241455078125, 164.89268493652..."
148,20240120_161207_jpg.rf.b811af46b8e7c338c1ffaab...,[]
149,DJI_0685-1-_JPG.rf.28e5acd6707444bdffa08e561f9...,[]


In [ ]:
df.to_csv('submit_i_hope0.csv', index=False)

In [ ]:
ex_sub = pd.read_csv('/content/sample_submission_with_errors.csv')
ex_sub.head(3)

,image_id,objects
0,20240120_153843-0-_jpg.rf.60b55d7dd594d0869bf7...,"[[""corrosion"", 45.85849989276216, 96.317499685..."
1,20240120_153843_jpg.rf.b24d413669fa57588901891...,[]
2,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,"[[""damage"", 152.5677603675233, 134.65200109050..."


In [ ]:
df_submit = ex_sub.copy()
df_submit.head(3)

,image_id,objects
0,20240120_153843-0-_jpg.rf.60b55d7dd594d0869bf7...,"[[""corrosion"", 45.85849989276216, 96.317499685..."
1,20240120_153843_jpg.rf.b24d413669fa57588901891...,[]
2,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,"[[""damage"", 152.5677603675233, 134.65200109050..."


In [ ]:
for i, row in df_submit.iterrows():
  cur_name = row['image_id']
  upd_val = pred_dict[cur_name]

  df_submit.at[i, 'objects'] = upd_val

In [ ]:
df_submit.head(3)

,image_id,objects
0,20240120_153843-0-_jpg.rf.60b55d7dd594d0869bf7...,"[[corrosion, 230.6865234375, 167.6171264648437..."
1,20240120_153843_jpg.rf.b24d413669fa57588901891...,[]
2,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,[]


In [ ]:
df_submit.to_csv('submit_i_hope00.csv', index=False, )


In [ ]:
df_submit

,image_id,objects
0,20240120_153843-0-_jpg.rf.60b55d7dd594d0869bf7...,"[[corrosion, 230.6865234375, 167.6171264648437..."
1,20240120_153843_jpg.rf.b24d413669fa57588901891...,[]
2,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,[]
3,DJI_0765-1-_JPG.rf.500d5a743566278e0be2d7cd60c...,[]
4,DJI_0987_JPG.rf.c2002d99afd301f941c19bd6e56a91...,[]
...,...,...
146,20240120_163650-0-_jpg.rf.6ff26d6832686600c6f1...,[]
147,20240120_163730-0-_jpg.rf.7139d077e629ae78b030...,"[[corrosion, 1811.530517578125, 141.6571197509..."
148,20240120_155211_mp4-0126_jpg.rf.5a9e67fd5bce79...,[]
149,20240120_155211_mp4-0084_jpg.rf.de984895faf812...,[]
